## Bibliotecas 

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";

# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="1";

In [ ]:
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, exposure
from skimage.transform import resize 
from skimage.io import imread_collection
from sklearn.model_selection import train_test_split
from skimage.feature import local_binary_pattern
import glob
import time
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import os
import pydicom
import pandas as pd
from skimage.feature import greycomatrix, greycoprops
from skimage.io import imread, imshow, imsave, imread_collection
from tqdm import tqdm
import cv2
%matplotlib inline

## Lendo a base 

In [ ]:
def lendo_imagens_dicom(path):
    slices = [imread(path + '/' + s) for s in os.listdir(path)] 
    return(slices)

In [ ]:
path = '/data/flavio/Camila/imagens/clahe/originais-clahe/lung-clahe/'
image_names = os.listdir(path)
covid = [imread(path+str(name)+'') for name in image_names]
print('The database has {} segmented images'.format(len(covid)))


In [ ]:
path = '/data/flavio/Camila/imagens/clahe/originais-clahe/normal_clahe/'
image_names = os.listdir(path)
normal = [imread(path+str(name)+'') for name in image_names]
print('The database has {} segmented images'.format(len(normal)))

In [ ]:
print(resize_i1[0].shape)

In [ ]:
#data_path = '/data/flavio/Camila/imagens/'
# Conjunto de imagens 
#covid = lendo_imagens_dicom(data_path+'covid_multi')
#lung = lendo_imagens_dicom(data_path+'lung_multi')
#print('Tamanho conjunto benigno: ', len(lung))
#print('Tamanho conjunto maligno: ', len(covid))

## Separando Conjunto 

In [ ]:
# Separando o conjunto de imagens 'Benigno' em 70% Treino e 30% Teste
benigno_treino_70, benigno_teste_30 = train_test_split(normal, test_size=0.3)
# Separando o conjunto de imagens 'Maligno' em 70% Treino e 30% Teste
maligno_treino_70, maligno_teste_30 = train_test_split(covid, test_size=0.3)

In [ ]:
print('Benigno Treino 70%: ', len(benigno_treino_70))
print('Maligno Treino 70%: ', len(maligno_treino_70))
print('Benigno Teste 30%: ', len(benigno_teste_30))
print('Maligno Teste 30%: ', len(maligno_teste_30))

In [ ]:
from tqdm import tqdm
class0_treino = []
for i in tqdm(benigno_treino_70): 
           im = cv2.resize((i),(256, 256))
           class0_treino.append(im)

In [ ]:
from tqdm import tqdm
class1_treino = []
for i in tqdm(maligno_treino_70): 
           im = cv2.resize((i),(256, 256))
           class1_treino.append(im)

In [ ]:
from tqdm import tqdm
class0_teste = []
for i in tqdm(benigno_teste_30): 
           im = cv2.resize((i),(256, 256))
           class0_teste.append(im)

In [ ]:
from tqdm import tqdm
class1_teste = []
for i in tqdm(maligno_teste_30): 
           im = cv2.resize((i),(256, 256))
           class1_teste.append(im)

## Normaizando as imagens 

In [ ]:
def scaler(imagem):
    sk  = MinMaxScaler(feature_range=(0, 255))
    sk.fit(imagem)
    new_imagem= sk.transform(imagem)
    return (new_imagem.astype(np.uint8))

## Salvando CSV 

In [ ]:
def csv_save_bm(matriz_b, matriz_m, name_file):  
    colunas = ['contrast','dissimilarity','homogeneity','correlation','asm','energy', 'classe']
    matriz_b_csv = pd.DataFrame(matriz_b,columns =colunas)
    matriz_m_csv = pd.DataFrame(matriz_m, columns=colunas)
    matriz_csv = pd.concat([matriz_b_csv, matriz_m_csv] )
    matriz_csv.to_csv(name_file+'.csv', index=False)
    #print('CSV : [ '+ name_file+ ' ] gerado com sucesso!')

## Features GLGM

In [ ]:
def glcm_features(glcm_matriz): 
    features = np.zeros((6))
    features[0] = (greycoprops(glcm_matriz, 'contrast')[0])
    features[1] = (greycoprops(glcm_matriz, 'dissimilarity')[0])
    features[2] = (greycoprops(glcm_matriz, 'homogeneity')[0])
    features[3] = (greycoprops(glcm_matriz, 'correlation')[0])
    features[4] = (greycoprops(glcm_matriz, 'ASM')[0])
    features[5] = (greycoprops(glcm_matriz, 'energy')[0])
    return(features)

[](http://)## Função GLCM


In [ ]:
def glcm_csv2(base_imagens, dis, ang, classe): 
    #matriz = cria_matrix(tam)
    matriz = []
    target = []
    #Aplicacao na base maligna
    for i, image in enumerate(base_imagens):
        image2 = image.copy()
        # Quando a imagem tem apenas 1 volume 
        if len(image2.shape) < 3:
            glcm = greycomatrix(scaler(image2), [dis], [ang], levels=256, symmetric = True, normed=True)
            matriz.append(glcm_features(glcm))
            target.append(classe)
        # Quando a imagem tem mais de 1 volume     
        else: 
            for j in range(image2.shape[0]): 
                # Adicionando os volumes na nova imagem 
                glcm = greycomatrix(scaler(image2[j,:,:]), [dis], [ang], levels=256, symmetric = True, normed=True)
                matriz.append(glcm_features(glcm))
                target.append(classe)
    print("terminou base imagens")

    # converter lista para np.array
    matriz_glcm = np.array(matriz)
    classe_glcm = np.array(target)
    #print(x.shape)
    #Juntar o X com o Y
    matriz_saida = np.zeros((matriz_glcm.shape[0], matriz_glcm.shape[1]+1))
    matriz_saida[0:,:-1] = matriz_glcm
    matriz_saida[0:,-1] = classe_glcm.reshape(-1)
    print('Shape: ', classe_glcm.shape)
    print('Shape matriz: ', matriz_saida.shape)
    return(matriz_saida)

## Executando descritor GLCM


In [ ]:
# ------------ Configurações --------------------------
# distância
distancia = 3
# âgulo
angulo = 90

#------------------ CSV TREINO -----------------------------------

#Executando o descritor -> benigno_treino_70 -> classe 0
treino_glcm_bt = glcm_csv2(class0_treino, distancia, angulo, 0)
#Executando o descritor -> maligno_treino_70 -> classe 1
treino_glcm_mt = glcm_csv2(class1_treino, distancia, angulo, 1)
#Nome do arquivo csv que será salvo para a combinação de parametros
nome_do_csv = '/data/flavio/Camila/csv_clahe/resultante/glcm/original/treino_lung_normal_sseg'+str(distancia)+'_angulo'+str(angulo)
#Salvar resultado Benigno e Maligno 
csv_save_bm(treino_glcm_bt, treino_glcm_mt, nome_do_csv)

#------------------ CSV TESTE -----------------------------------
    
#Executando o descritor -> benigno_teste_30 -> classe 0
teste_glcm_bt = glcm_csv2(class0_teste, distancia, angulo,  0)
#Executando o descritor -> maligno_teste_30 -> classe 1
teste_glcm_mt = glcm_csv2(class1_teste, distancia, angulo, 1)
#Nome do arquivo csv que será salvo para a combinação de parametros
nome_do_csv = '/data/flavio/Camila/csv_clahe/resultante/glcm/original/teste_lung_normal_sseg'+str(distancia)+'_angulo'+str(angulo)
#Salvar resultado Benigno e Maligno 
csv_save_bm(teste_glcm_bt, teste_glcm_mt, nome_do_csv)